## Word Vectorisation Notebook

In [1]:
import os
import urllib.request
import numpy as np
import zipfile
import tensorflow as tf
import collections
import random
import h5py as h5py


from keras.models import Sequential,Model
from keras.optimizers import RMSprop
from keras.layers import Embedding,LSTM,Dense,Lambda,merge,Input
from keras.callbacks import TensorBoard,ModelCheckpoint,Callback
from keras import backend as K

/Applications/anaconda/envs/nlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists("./downloads/"+filename):
    filename, _ = urllib.request.urlretrieve(url + filename, "./downloads/"+filename)
  return filename

def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile("./downloads/"+filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

# Step 3: Function to generate a training batch for the skip-gram model.

filename = maybe_download('text8.zip')

In [3]:
vocabulary = read_data(filename)
print('Number of words: ', len(vocabulary))

def generate_batches(data, size, contextWidth, negativeSize):
  cHalfWidth = int(contextWidth/2)
  words = []
  contexts = []
  negatives = []
  index = random.sample(range(cHalfWidth,len(data)-cHalfWidth),size)
  for z in index:
      context = []
      for m in range(-cHalfWidth,cHalfWidth+1):
        if m == 0:
          words.append([data[z]])
        else: 
          context.append(data[z+m])
      contexts.append(context)
      negatives.append(random.sample(data,negativeSize))
  return([np.array(words),np.array(contexts),np.array(negatives)],[np.array([1]*size),np.array([[0]*negativeSize]*size)])

Number of words:  17005207


In [4]:
vocabulary_size = 5000
data_index = 0
batch_size = 128
wordvec_dim = 32

skip_window = 3       # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.
context_half = 3
context_size = context_half*2

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
neg_size = 5    # Number of negative examples to sample.


In [5]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.

In [15]:
with open('./logs/word2vec_label.tsv', 'w') as fr:
    for i in range(5000):
        fr.write(reverse_dictionary[i]+'\n')

In [6]:
X,Y = generate_batches(data, 500000, context_size, neg_size)
vX, vY = generate_batches(data, 5000, context_size, neg_size)

word = Input(shape=(1,))
context = Input(shape=(context_size,))
negSamples = Input(shape=(neg_size,))

word2vec = Embedding(input_dim=vocabulary_size,output_dim=wordvec_dim, embeddings_initializer='glorot_normal', name='word2vec')

vec_word = word2vec(word)
vec_context = word2vec(context)
vec_negSamples = word2vec(negSamples)
cbow = Lambda(lambda x: K.mean(x, axis=1))(vec_context)

word_context = merge([vec_word, cbow], mode='dot')
negative_context = merge([vec_negSamples, cbow], mode='dot', concat_axis=-1)

model = Model(input=[word,context,negSamples], output=[word_context,negative_context])
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
#model.summary()
tensorboard = TensorBoard(log_dir='./logs/wordvec', 
  batch_size=500, histogram_freq=1, write_images=True, write_grads=False, write_graph=True, embeddings_freq=1)
model_checkpoint = ModelCheckpoint('./logs/model.hdf5')
model.fit(X,Y,epochs=50,batch_size=500,callbacks=[model_checkpoint,tensorboard], validation_data=(vX,vY))

/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outpu

Train on 500000 samples, validate on 5000 samples
Epoch 1/50
500000/500000 [==============================] - 6s 11us/step - loss: 0.5484 - merge_1_loss: 0.3796 - merge_2_loss: 0.1688 - merge_1_acc: 0.3976 - merge_2_acc: 0.2317 - val_loss: 0.4821 - val_merge_1_loss: 0.2839 - val_merge_2_loss: 0.1982 - val_merge_1_acc: 0.5208 - val_merge_2_acc: 0.2262
Epoch 2/50
500000/500000 [==============================] - 6s 12us/step - loss: 0.4701 - merge_1_loss: 0.2741 - merge_2_loss: 0.1960 - merge_1_acc: 0.5188 - merge_2_acc: 0.2274 - val_loss: 0.4622 - val_merge_1_loss: 0.2688 - val_merge_2_loss: 0.1935 - val_merge_1_acc: 0.5196 - val_merge_2_acc: 0.2308
Epoch 3/50
500000/500000 [==============================] - 6s 12us/step - loss: 0.4537 - merge_1_loss: 0.2609 - merge_2_loss: 0.1928 - merge_1_acc: 0.5404 - merge_2_acc: 0.2271 - val_loss: 0.4502 - val_merge_1_loss: 0.2580 - val_merge_2_loss: 0.1922 - val_merge_1_acc: 0.5466 - val_merge_2_acc: 0.2314
Epoch 4/50
500000/500000 [===============

Epoch 28/50
500000/500000 [==============================] - 6s 13us/step - loss: 0.3683 - merge_1_loss: 0.1978 - merge_2_loss: 0.1705 - merge_1_acc: 0.6898 - merge_2_acc: 0.2286 - val_loss: 0.4111 - val_merge_1_loss: 0.2335 - val_merge_2_loss: 0.1776 - val_merge_1_acc: 0.6264 - val_merge_2_acc: 0.2266
Epoch 29/50
500000/500000 [==============================] - 7s 15us/step - loss: 0.3678 - merge_1_loss: 0.1973 - merge_2_loss: 0.1704 - merge_1_acc: 0.6913 - merge_2_acc: 0.2288 - val_loss: 0.4118 - val_merge_1_loss: 0.2360 - val_merge_2_loss: 0.1758 - val_merge_1_acc: 0.6212 - val_merge_2_acc: 0.2268
Epoch 30/50
500000/500000 [==============================] - 7s 14us/step - loss: 0.3672 - merge_1_loss: 0.1969 - merge_2_loss: 0.1703 - merge_1_acc: 0.6921 - merge_2_acc: 0.2288 - val_loss: 0.4120 - val_merge_1_loss: 0.2346 - val_merge_2_loss: 0.1773 - val_merge_1_acc: 0.6260 - val_merge_2_acc: 0.2222
Epoch 31/50
500000/500000 [==============================] - 7s 14us/step - loss: 0.3668